# Memoris project

## Imports

#### Internal modules

In [ ]:
from core.visual import Borehole3D, striplog_legend_to_omf_legend 
from utils.lexicon_memoris import LEG_CONTAMINATION_LEV
from utils.io import dataframe_viewer, dict_viewer
from utils.orm import orm_boreholes_from_dataframe
from utils.visual import legend_from_attributes, find_new_word_for_legend
from core.project import Project
from core.orm import BoreholeOrm, PositionOrm, Base 
from utils.config import DEFAULT_LITHO_LEXICON, DEFAULT_LITHO_LEGEND, DEFAULT_BOREHOLE_LEXICON, \
DEFAULT_BOREHOLE_LEGEND, DEFAULT_POL_LEXICON

#### External modules

In [ ]:
import matplotlib.pyplot as plt
import os
import re
import pandas as pd
import numpy as np
from striplog import Component, Decor, Legend, Lexicon, Striplog
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

## Reading of CSV files in a dir

In [ ]:
data_dir = ROOT_DIR + '/CF_data/Result_traitem/fusion_finale/filtered/' 

In [ ]:
litho_file = data_dir + 'Lithologies.csv'
sample_file = data_dir + 'Samples.csv'
bh_file = data_dir + 'Boreholes.csv'

In [ ]:
litho = pd.read_csv(litho_file, sep=',')
sample = pd.read_csv(sample_file, sep=',')
borehole = pd.read_csv(bh_file, sep=',')

In [ ]:
dataframe_viewer(borehole, rows=5, un_val='ID'), dataframe_viewer(litho, rows=5, un_val='ID'), dataframe_viewer(sample, rows=5, un_val='ID')

In [ ]:
litho.loc[1573, 'Litho_base'] = 4.5

In [ ]:
data_list = [litho, sample]
bh_id='F81'
cols=['Long_for', '_top', '_base']

for data in data_list:
    view_cols = []
    for col in data.columns:
        for c in cols:
            if re.search(c, col, re.I):
                view_cols.append(col)
                
    dataframe_viewer(data.query(f"ID=='{bh_id}'")[view_cols], rows=3, cols=16)

## Create pollutants list from the dataframe columns name

In [ ]:
pollutants = []
skipped = []
for i, c in enumerate(sample.columns):
    if c in DEFAULT_POL_LEXICON.abbreviations.keys() or c in DEFAULT_POL_LEXICON.abbreviations.values():
        #print(f"{i}: {c}")
        pollutants.append(c)
    else:
        skipped.append(c)
print(pollutants)

## Create Boreholes and components from dataframes

In [ ]:
data_list = [litho[:500], sample[:500]]

In [ ]:
boreholes_orm, components, link_intv_comp = orm_boreholes_from_dataframe(data_list, default_z=102, id_col='ID', 
                                                bh_type_col='Type', date_col='Date_for',
                                                length_col = 'Long_for', diameter_col='Diam_for',
                                                sample_type_col='Type_ech', sample_id_col='ID_ech',
                                                attributes=['lithology']+pollutants, 
                                                symbols={'lithology':{'lexicon':DEFAULT_LITHO_LEXICON}}
                                                )

In [ ]:
len(boreholes_orm), len(components)

In [ ]:
boreholes_orm # list of core.BoreholeOrm objects

In [ ]:
components # dict of striplog.Component objects

## Create a dictionary containing legends for each representation attribute

In [ ]:
legend_dict = legend_from_attributes([('borehole_type', DEFAULT_BOREHOLE_LEGEND), 
                                      ('lithology', DEFAULT_LITHO_LEGEND)] + pollutants)

## Create a database for the project

In [ ]:
DB_name = '../tmp_files/memoris_db.db'

if os.path.exists(DB_name):
    os.remove(DB_name)

engine = create_engine(f"sqlite:///{DB_name}", echo=False)

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

## create a project

In [ ]:
p = Project(session, name='Memoris_project', legend_dict=legend_dict, lexicon=DEFAULT_LITHO_LEXICON)

### Insert boreholes and components into the project

In [ ]:
p.add_components(components)

In [ ]:
for bhorm in boreholes_orm:
    p.add_borehole(bhorm)

In [ ]:
p.add_link_components_intervals(link_intv_comp)

### Validate modifications 

In [ ]:
p.commit()

In [ ]:
p.refresh(update_3d=True)

In [ ]:
len(p.boreholes_3d)

In [ ]:
session.close()

## Project legend dict

In [ ]:
attributes = list(legend_dict.keys())
print(attributes)

In [ ]:
n = 1
print('Chosen attribute is :', attributes[n])
p.repr_attribute = attributes[n]

In [ ]:
p.legend_dict[p.repr_attribute]['legend']

In [ ]:
p.legend_dict[attributes[n]]['cmap']

In [ ]:
find_new_word_for_legend(p.legend_dict, p.repr_attribute)

## Visualization

In [ ]:
bh_name = '201'

### Plot a log for one borehole, according to the representation attribute

In [ ]:
p.boreholes_3d[bh_name]#.attrib_components()

In [ ]:
p.boreholes_3d[bh_name].attrib_components()

In [ ]:
p.plot_log(bh_name=bh_name)

### Localization of project boreholes

In [ ]:
p.plot_map()

### 3D display of one borehole

In [ ]:
from utils.orm import create_bh3d_from_bhorm, interval_orm_to_interval

In [ ]:
def get_interval_list(bh_orm, attribute=None, project=None):
    """create a list of interval from a boreholeORM object, based on an attribute if not None

    Parameters
    -------------
    bh_orm: boreholeOrm object
        A BoreholeOrm object from which intervals matching the attribute will be listed
    attribute: str
        The attribute of components to search in components associated with the borehole intervals
    project: Project object

    Returns
    ---------
    interval_list: list
        List of Interval objects for each type of interval
    max_depth: float
        Borehole's maximum depth for all intervals

    """

    max_depth = None
    interval_list = []
    for int_id, intv_orm in bh_orm.intervals.items():
        for c in intv_orm.components:
            interval_attributes = [i for i in eval(c.description).keys()]
            if interval_attributes[0] == 'pollutant':
                interval_attributes = [eval(c.description)['pollutant']]

            if (attribute in interval_attributes) or (attribute is None):
                interval_list.append(interval_orm_to_interval(intv_orm, project))

            if 'borehole_type' in interval_attributes:
                max_depth = intv_orm.base.middle
    assert max_depth is not None
    return interval_list, max_depth

In [ ]:
n = 'zinc'
if isinstance(n, str):
    n = attributes.index(n)

print('Chosen attribute is :', attributes[n])

In [ ]:
p.repr_attribute = attributes[n]
self = p
bhs_3d = {}
for bh in self.boreholes_orm.values():
    if len(get_interval_list(bh, self.repr_attribute)[0])>0:
        z_ref = get_interval_list(bh, 'borehole_type')[0][0].top.z
        bhs_3d.update({bh.id : create_bh3d_from_bhorm(bh, z_ref=z_ref, attribute=self.repr_attribute, 
                                                      legend_dict=self.legend_dict)})

In [ ]:
bhs_3d

In [ ]:
pause

### 3D display of project boreholes

In [ ]:
n = 5
print('Chosen attribute is :', attributes[n])

In [ ]:
p.repr_attribute = attributes[n]

In [ ]:
p.legend_dict[p.repr_attribute]['legend']

In [ ]:
bg_color="white" # ["royalblue", "aliceblue"]
p.plot_3d(labels_size=15, bg_color=bg_color, window_size=(800, 600), jupyter_backend='panel', 
          verbose=True)